# Train CIFAR with the `policy` module

Let's install the latest version of Poutyne (if it's not already) and import all the needed packages.

In [1]:
%pip install --upgrade poutyne

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18

from poutyne import Model, OptimizerPolicy, one_cycle_phases

Requirement already up-to-date: poutyne in /home/fredy/.venv/base/lib/python3.8/site-packages (1.0.0)
Note: you may need to restart the kernel to use updated packages.


## Training constant
But first, let's set the training constants, the CUDA device used for training if one is present, we set the batch size (i.e. the number of elements to see before updating the model) and the number of epochs (i.e. the number of times we see the full dataset).

In [2]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

batch_size = 1024
epochs = 5

# Load the data

In [3]:
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])

In [4]:
root = "datasets"
train_ds = datasets.CIFAR10(root, train=True, transform=train_transform, download=True)
val_ds = datasets.CIFAR10(root, train=False, transform=val_transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_dl = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8
)
val_dl = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=8
)

# The model
We'll train a simple `resNet-18` network.
This takes a while without GPU but is pretty quick with GPU.

In [6]:
def get_module():
    model = resnet18(pretrained=False)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.fc = nn.Linear(512, 10)
    return model


# Training without the `policies` module

In [7]:
pytorch_network = get_module()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_network.parameters(), lr=0.01)

model = Model(
    pytorch_network,
    optimizer,
    criterion,
    batch_metrics=["acc"],
)
model = model.to(device)

history = model.fit_generator(
    train_dl,
    val_dl,
    epochs=epochs,
)

Epoch: 1/5 Step: 49/49 100.00% |█████████████████████████|8.71s loss: 2.115406 acc: 22.852000 val_loss: 1.870449 val_acc: 32.520000
Epoch: 2/5 Step: 49/49 100.00% |█████████████████████████|8.59s loss: 1.789251 acc: 34.850000 val_loss: 1.664176 val_acc: 39.490000
Epoch: 3/5 Step: 49/49 100.00% |█████████████████████████|8.58s loss: 1.638004 acc: 40.164000 val_loss: 1.560369 val_acc: 42.310000
Epoch: 4/5 Step: 49/49 100.00% |█████████████████████████|8.76s loss: 1.558112 acc: 43.102000 val_loss: 1.494177 val_acc: 45.360000
Epoch: 5/5 Step: 49/49 100.00% |█████████████████████████|8.70s loss: 1.493087 acc: 45.810000 val_loss: 1.469608 val_acc: 46.800000


# Training with the `policies` module

In [8]:
steps_per_epoch = len(train_dl)
steps_per_epoch

49

In [9]:
pytorch_network = get_module()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_network.parameters(), lr=0.01)

model = Model(
    pytorch_network,
    optimizer,
    criterion,
    batch_metrics=["acc"],
)
model = model.to(device)

policy = OptimizerPolicy(
    one_cycle_phases(epochs * steps_per_epoch, lr=(0.01, 0.1, 0.008)),
)
history = model.fit_generator(
    train_dl,
    val_dl,
    epochs=epochs,
    callbacks=[policy],
)

Epoch: 1/5 Step: 49/49 100.00% |█████████████████████████|8.62s loss: 1.848409 acc: 33.260000 val_loss: 1.755679 val_acc: 42.600000
Epoch: 2/5 Step: 49/49 100.00% |█████████████████████████|8.73s loss: 1.380307 acc: 50.424000 val_loss: 1.286768 val_acc: 54.810000
Epoch: 3/5 Step: 49/49 100.00% |█████████████████████████|8.61s loss: 1.170797 acc: 58.460000 val_loss: 1.094710 val_acc: 60.010000
Epoch: 4/5 Step: 49/49 100.00% |█████████████████████████|8.62s loss: 0.970011 acc: 65.824000 val_loss: 0.938677 val_acc: 67.060000
Epoch: 5/5 Step: 49/49 100.00% |█████████████████████████|8.80s loss: 0.809022 acc: 71.444000 val_loss: 0.871968 val_acc: 69.920000
